In [1]:
import sys
import os
from pathlib import Path
import torch
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.DEVICE = 'cpu'
args.task = 'temporal'
args.metric = 'temporal_acc'
args.SoftEva = True
args = FormulateArgs(args)
args.N_test = 10
args.e_test = 0.1

In [3]:
args.augment = False


In [ ]:
state_dict = model.state_dict()
for name in state_dict:
    if name.endswith('rt_') or name.endswith('beta') or name.endswith('theta_'):
        print(name, state_dict[name])

In [8]:
results = torch.zeros([15,10,2])

for seed in range(10):
    for ds in range(15):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

        modelname = f"pLF_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        # print(modelname)
        
        if os.path.isfile(f'./models/{modelname}'):                
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.UpdateArgs(args)
            print(model.DEVICE)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            with torch.no_grad():
                state_dict = model.state_dict()
                accs_valid = []
                accs_test = []
                for _ in range(args.N_test):
                    for name in state_dict:
                        if name.endswith('rt_') or name.endswith('beta') or name.endswith('theta_'):
                            value = state_dict[name].to(args.DEVICE)
                            nosie = (((torch.rand(value.shape) * 2.) - 1.) * args.e_test + 1.).to(args.DEVICE)
                            state_dict[name] = value * nosie 
                            # print(name, mean.shape)
                    model.load_state_dict(state_dict)
        
                    accs_valid.append(evaluator(model, X_valid, y_valid))
                    accs_test.append(evaluator(model, X_test,  y_test))
                print(accs_valid, accs_test)
                results[ds,seed,0] = np.mean(accs_valid)
                results[ds,seed,1] = np.mean(accs_test)
        else:
            results[ds,seed,:] = float('nan')

cpu
[0.94, 0.88, 0.93, 0.63, 0.65, 0.73, 0.65, 0.85, 0.79, 0.68] [0.96, 0.89, 0.89, 0.51, 0.63, 0.62, 0.71, 0.84, 0.8, 0.64]
cpu
[0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47] [0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46]
cpu
[0.48, 0.48, 0.52, 0.46, 0.58, 0.5, 0.5, 0.45, 0.55, 0.54] [0.53, 0.53, 0.52, 0.54, 0.46, 0.51, 0.45, 0.48, 0.5, 0.44]
cpu
[0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53] [0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54]
cpu
[0.4444444444444444, 0.4444444444444444, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222, 0.4222222222222222] [0.5054945054945055, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725, 0.4725274725274725]
cpu
[0.7333333333333333, 0.7333333333333333, 0.5888888888888889, 0.5555555555555556, 0.6111111

In [ ]:
results

In [9]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/1_non_var_test_top_{N_selected_seeds}_baseline_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [ ]:
print(datainfo)

In [1]:
results.shape

NameError: name 'results' is not defined

In [5]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,0], N_selected_seeds, dim=1, largest=True, sorted=True)
selected_results = results.gather(1, indices.unsqueeze(-1).expand(-1, -1, 2))
selected_test_mean = selected_results[:,:,-1].mean(1)
selected_test_std = selected_results[:,:,-1].std(1)
selected_test_results = torch.cat([selected_test_mean.unsqueeze(-1), selected_test_std.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/top_{N_selected_seeds}_LPF_acc.txt", selected_test_results.numpy(), delimiter="\t", fmt='%.5f')


In [1]:
results

NameError: name 'results' is not defined